## Using latent semantic indexing on labor categories

This is an attempt to use gensim's [Latent Semantic Indexing](https://radimrehurek.com/gensim/models/lsimodel.html) functionality with contract data, providing us with a way to find contract rows whose labor categories are similar to one we're looking at. Then we'll combine that data with some other dimensions from the contract rows, like price and minimum experience, and finally use a [K Nearest Neighbors](https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm) algorithm to help us find comparables for any given criteria.

Code is largely based off the [Making an Impact with Python Natural Language Processing Tools](https://www.youtube.com/watch?v=jSdkFSg9oW8) Pycon 2016 tutorial, specifically its [LSI with Gensim](https://github.com/totalgood/twip/blob/master/docs/notebooks/09%20Features%20--%20LSI%20with%20Gensim.ipynb) notebook.

In [1]:
import pandas as pd

rows = pd.read_csv('../data/hourly_prices.csv', index_col=False, thousands=',')

In [2]:
from gensim.models import LsiModel, TfidfModel
from gensim.corpora import Dictionary

/Users/atulvarma/Documents/explorations/calc-metrics-fun/venv/lib/python3.5/site-packages/gensim/utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


We'll build a vocabulary off the labor categories in the contract rows, and then build a [term frequency–inverse document frequency](https://en.wikipedia.org/wiki/Tf%E2%80%93idf) matrix off it.

In [3]:
vocab = Dictionary(rows['Labor Category'].str.split())

In [4]:
tfidf = TfidfModel(id2word=vocab, dictionary=vocab)

In [5]:
bows = rows['Labor Category'].apply(lambda x: vocab.doc2bow(x.split()))

In [6]:
vocab.token2id['engineer']

650

In [7]:
vocab[0]

'Manager'

In [8]:
dict([(vocab[i], round(freq, 2)) for i, freq in tfidf[bows[0]]])

{'Manager': 0.65, 'Project': 0.76}

Here we'll build a LSI model that places each labor category into a 5-dimensional vector.

In [9]:
lsi = LsiModel(tfidf[bows], num_topics=5, id2word=vocab, extra_samples=100, power_iters=2)

In [10]:
len(vocab)

6944

In [11]:
topics = lsi[bows]
df_topics = pd.DataFrame([dict(d) for d in topics], index=bows.index, columns=range(5))

In [12]:
lsi.print_topic(1, topn=5)

'-0.511*"Consultant" + -0.333*"Analyst" + 0.329*"Manager" + 0.272*"Project" + -0.255*"Senior"'

This part is a bit weird: we're extending our vectors with information about the price and minimum experience of each contract row, semi-normalizing the data so that they don't "overwhelm" the importance of the LSI dimensions when calculating distances between points.

I have no idea if this is actually legit.

In [13]:
PRICE_COEFF = 1 / 500.0
XP_COEFF = 1 / 10.0

df_topics['Price'] = (rows['Year 1/base'] * PRICE_COEFF).fillna(0)
df_topics['Experience'] = (rows['MinExpAct'] * XP_COEFF).fillna(0)

Now we'll use a [K Nearest Neighbors](https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm) algorithm to make it easy for us to find vectors that are nearby.

In [14]:
from sklearn.neighbors import NearestNeighbors

df_topics = df_topics.fillna(0)
neigh = NearestNeighbors(n_neighbors=5)
neigh.fit(df_topics)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=1, n_neighbors=5, p=2, radius=1.0)

In [15]:
neigh.kneighbors(df_topics.ix[0], return_distance=False)

/Users/atulvarma/Documents/explorations/calc-metrics-fun/venv/lib/python3.5/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


array([[  0,  17,  18, 104, 256]])

Here's where things potentially become useful: we'll create a function that takes a labor category, price, and experience, and returns a list of comparables from our vector space.

In [16]:
def get_neighbors(labor_category, price, experience):
    vector = []
    topic_values = lsi[tfidf[vocab.doc2bow(labor_category.split())]]
    vector.extend([v[1] for v in topic_values])
    vector.extend([price * PRICE_COEFF, experience * XP_COEFF])
    
    neighbors = list(neigh.kneighbors(vector, return_distance=False)[0])
    return pd.DataFrame([rows.loc[i] for i in neighbors], index=neighbors)

get_neighbors('Awesome Project Manager', 80, 5)

/Users/atulvarma/Documents/explorations/calc-metrics-fun/venv/lib/python3.5/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


,Labor Category,Year 1/base,Year 2,Year 3,Year 4,Year 5,Education,MinExpAct,Bus Size,Location,COMPANY NAME,CONTRACT .,Schedule,SIN NUMBER,Contract Year,Begin Date,End Date,CurrentYearPricing
44548,Project Director 3/Field Director 3,55.68,55.68,55.68,55.68,55.68,Bachelors,6.0,S,Both,"Desert Archaeology, Inc�",GS-10F-161AA,Environmental,899-7,4,5/3/2013,5/2/2018,55.68
44547,Project Director 3/Field Director 3,55.68,55.68,55.68,55.68,55.68,Bachelors,6.0,S,Both,"Desert Archaeology, Inc�",GS-10F-161AA,Environmental,899-1,4,5/3/2013,5/2/2018,55.68
24335,Project Director 1,101.15,103.27,105.44,107.66,109.92,Bachelors,5.0,O,Both,"National Alliance for Hispanic Health, The",GS-10F-088CA,MOBIS,874-1,2,5/13/2015,5/12/2020,103.27
31639,Project Director,83.75,83.75,83.75,83.75,83.75,Bachelors,5.0,S,Contractor Site,"Advocates For Human Potential, Inc.",GS-10F-0148P,MOBIS,"874-1, 874-2, 874-3, 874-7",3,1/16/2004,1/15/2019,83.75
31638,Project Director,83.75,83.75,83.75,83.75,83.75,Bachelors,5.0,S,Both,"Advocates For Human Potential, Inc.",GS-10F-0148P,MOBIS,874-1,3,1/16/2004,1/15/2019,83.75
